## **VENTE**



In [ ]:
# Création d'une vue avec le total des quantité commandé sur une année glissante.
CREATE VIEW quantity_now AS
    SELECT
    p.productLine,
    YEAR(orderDate) AS year,
    MONTH(orderDate) AS month,
    SUM(od.quantityOrdered) AS total_quantity
FROM
    products AS p
INNER JOIN
    productlines AS pl ON pl.productLine = p.productLine
INNER JOIN
    orderdetails AS od ON od.productCode = p.productCode
INNER JOIN
    orders ON orders.orderNumber = od.orderNumber
WHERE
    orderDate >= DATE_SUB((SELECT MAX(orderDate) FROM orders), INTERVAL 11 MONTH)
GROUP BY
    p.productLine, year, month;



# Création d'une vue avec le total des quantité commandé sur une année glissante a-2.
CREATE VIEW quantity_last_year AS
    SELECT
    p.productLine,
    YEAR(orderDate) AS year,
    MONTH(orderDate) AS month,
    SUM(od.quantityOrdered) AS total_quantity
FROM
    products AS p
INNER JOIN
    productlines AS pl ON pl.productLine = p.productLine
INNER JOIN
    orderdetails AS od ON od.productCode = p.productCode
INNER JOIN
    orders ON orders.orderNumber = od.orderNumber
WHERE
    orderDate >= DATE_SUB((SELECT MAX(orderDate) FROM orders), INTERVAL 23 MONTH)
    AND orderDate < DATE_SUB((SELECT MAX(orderDate) FROM orders), INTERVAL 11 MONTH)
GROUP BY
    p.productLine, year, month;


# Jointure entre les deux vue, avec création d'une colonne variation permettant de connaitre le taux de variation entre les deux années glissantes.
SELECT
    qn.productLine,
    qn.year,
    qn.month,
    qn.total_quantity,
    qly.total_quantity AS quantity_last_year,
    ROUND(((qn.total_quantity-qly.total_quantity)/qly.total_quantity)*100, 2) AS variation
    quantity_now AS qn
LEFT JOIN
    quantity_last_year AS qly ON qn.productLine = qly.productLine AND qly.month = qn.month
    year DESC, month DESC

# FINANCE

In [ ]:
#Question 1 :

# Création d'une vue pour permettant de connaitre le chiffre d'affaire des deux derniers mois.
CREATE VIEW two_month_amount AS
SELECT
    c.country,
    SUM(amount) AS chiffre_affaire
FROM
    payments p
JOIN
    customers c ON p.customerNumber = c.customerNumber
WHERE
    YEAR(paymentDate) = (SELECT YEAR(MAX(paymentDate)) FROM payments) AND
    MONTH(paymentDate) BETWEEN MONTH((SELECT MAX(paymentDate) FROM payments)) - 2 AND MONTH((SELECT MAX(paymentDate) FROM payments)) - 1
GROUP BY
    c.country;

#Question 2 :

# Création d'une vue permettant de récuperer les commandes non payés.
CREATE VIEW on_hold_status AS
SELECT
    c.customerNumber,
    c.customerName,
    pay.paymentDate,
    o.status,
    o.comments
FROM
    customers c
JOIN
    orders o ON c.customerNumber = o.customerNumber
JOIN
    payments pay ON o.customerNumber = pay.customerNumber
WHERE
    status = 'On Hold' AND paymentDate = '2023-06-29' ; # on exclus les commandes envoyé, résolue et annulé pour ne garder que les commandes non payés.

# PARTIE RESSOURCE HUMAINE


In [ ]:
# Création d'une vue permettant d'avoir, à partir de la dernière date mis à jour, les 2 employés ayant fais le plus de chiffre d'affaire le mois dernier.

CREATE VIEW best_employee AS
SELECT
    c.salesRepEmployeeNumber as employee_ID,
    firstName,
    lastName,
    SUM(p.amount) AS total_turnover,
    YEAR(p.paymentDate) AS year,
    MONTH(p.paymentDate) AS month
FROM
    payments as p
INNER JOIN
    customers as c ON p.customerNumber = c.customerNumber
INNER JOIN
    employees as e ON c.salesRepEmployeeNumber = e.employeeNumber
WHERE
   YEAR(p.paymentDate) = (SELECT YEAR(MAX(paymentDate)) FROM payments) AND
   MONTH(p.paymentDate) = (SELECT MONTH(MAX(paymentDate)) FROM payments) - 1
GROUP BY
    year, MONTH(p.paymentDate), employee_ID, firstName, lastName
ORDER BY
    total_turnover DESC
LIMIT 2;

# LOGISTIQUE

In [ ]:
# Création d'une vue permettant de connaitre le stock des 5 produits les plus commandés grâce à l'utilisation d'une sous-requête.

CREATE VIEW stock_best_ordered AS
SELECT
   *
FROM (
    SELECT p.productName,SUM(od.quantityOrdered) AS totalQuantityOrdered, p.quantityInStock,
    RANK() OVER (ORDER BY SUM(od.quantityOrdered) DESC) AS product_rank
    FROM orderdetails AS od
    INNER JOIN products AS p ON p.productCode = od.productCode
    GROUP BY p.productName,p.quantityInStock
) AS ranked_products
WHERE product_rank <= 5;